# Read_data & EDA

In [3]:
#package load
import pandas as pd
import numpy as np
import os
dicpath = os.getcwd()

In [16]:
lgbm_train01 = pd.read_csv('lgbm_train_01.csv',index_col=0)
lgbm_train02 = pd.read_csv('lgbm_train_02.csv',index_col=0).sort_values('acc_id')


# Feature Engineering

In [17]:
cat_fea = [x for x in list(lgbm_train01) if 'cat' in x]
num_fea = lgbm_train01.drop(['label','acc_id'],axis=1).drop(columns = cat_fea)
num_fea_col = num_fea.columns
num_fea = num_fea.fillna(0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(num_fea)
scaled_x = pd.DataFrame(scaler.transform(num_fea))
scaled_x.columns = num_fea_col


In [18]:
scaled_x['acc_id'] = lgbm_train01["acc_id"]
scaled_x['label'] = lgbm_train01["label"]

# modeling

In [20]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout,Activation
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import KFold, StratifiedKFold
from keras.callbacks import EarlyStopping
from keras.models import load_model
num_folds = 5

feats = [f for f in scaled_x.columns if f not in ['label','acc_id']]
folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)

result = []
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(scaled_x[feats], scaled_x['label'])):


    X_train, y_train = scaled_x[feats].iloc[train_idx], scaled_x['label'].iloc[train_idx]
    X_valid, y_valid = scaled_x[feats].iloc[valid_idx], scaled_x['label'].iloc[valid_idx]

    #data_type check
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    #labels -> 4 categories
    y_train = np_utils.to_categorical(y_train,4)
    y_valid = np_utils.to_categorical(y_valid,4)

    #model
    model = Sequential()
    model.add(Dense(512,input_shape=(773,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(4))
    model.add(Activation('softmax'))
    
    
    #early_stop
    early_stopping_callback = EarlyStopping(monitor='loss',patience=5)
    
    #model const
    model.compile(loss = 'categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

    #train
    hist = model.fit(X_train,y_train,epochs=80,batch_size=64,callbacks=[early_stopping_callback])
    model.save('prob_nn_'+str(n_fold)+'.h5')
    #valid
    score = model.evaluate(X_valid,y_valid,verbose=1)
    print('loss=',score[0])
    print('accuracy=',score[1])
    result.append(score[1])


Epoch 1/80
80000/80000 [==============================] - 9s 110us/step - loss: 0.8718 - acc: 0.6254
Epoch 2/80
80000/80000 [==============================] - 8s 104us/step - loss: 0.7929 - acc: 0.6506
Epoch 3/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7692 - acc: 0.6599
Epoch 4/80
80000/80000 [==============================] - 9s 107us/step - loss: 0.7539 - acc: 0.6670
Epoch 5/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7462 - acc: 0.6684
Epoch 6/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7425 - acc: 0.6727
Epoch 7/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7351 - acc: 0.6751
Epoch 8/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7285 - acc: 0.6771
Epoch 9/80
80000/80000 [==============================] - 9s 106us/step - loss: 0.7241 - acc: 0.6783
Epoch 10/80
80000/80000 [==============================] - 9s 106us/step - loss: 0.7191 - a

20000/20000 [==============================] - 1s 32us/step
loss= 0.755049827003479
accuracy= 0.7083
Epoch 1/80
80000/80000 [==============================] - 9s 110us/step - loss: 0.8671 - acc: 0.6277
Epoch 2/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7896 - acc: 0.6525
Epoch 3/80
80000/80000 [==============================] - 9s 112us/step - loss: 0.7677 - acc: 0.6596
Epoch 4/80
80000/80000 [==============================] - 8s 106us/step - loss: 0.7576 - acc: 0.6650
Epoch 5/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7459 - acc: 0.6705
Epoch 6/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7372 - acc: 0.6733
Epoch 7/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7276 - acc: 0.6771
Epoch 8/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7262 - acc: 0.6802
Epoch 9/80
80000/80000 [==============================] - 8s 105us/step - loss: 0.7211 - ac

20000/20000 [==============================] - 1s 30us/step
loss= 0.7526461999416352
accuracy= 0.7095
Epoch 1/80
80000/80000 [==============================] - 8s 101us/step - loss: 0.8751 - acc: 0.6275
Epoch 2/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7916 - acc: 0.6534
Epoch 3/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7701 - acc: 0.6626
Epoch 4/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7593 - acc: 0.6646
Epoch 5/80
80000/80000 [==============================] - 8s 97us/step - loss: 0.7490 - acc: 0.6707
Epoch 6/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7419 - acc: 0.6721
Epoch 7/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7342 - acc: 0.6741
Epoch 8/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7294 - acc: 0.6781
Epoch 9/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7244 - acc: 0.68

20000/20000 [==============================] - 1s 31us/step
loss= 0.7247529532432556
accuracy= 0.7076
Epoch 1/80
80000/80000 [==============================] - 8s 102us/step - loss: 0.8649 - acc: 0.6268
Epoch 2/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7910 - acc: 0.6553
Epoch 3/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7678 - acc: 0.6631
Epoch 4/80
80000/80000 [==============================] - 8s 97us/step - loss: 0.7539 - acc: 0.6669
Epoch 5/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7468 - acc: 0.6702
Epoch 6/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7386 - acc: 0.6721
Epoch 7/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7318 - acc: 0.6753
Epoch 8/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7270 - acc: 0.6782
Epoch 9/80
80000/80000 [==============================] - 8s 96us/step - loss: 0.7231 - acc: 0.67

20000/20000 [==============================] - 1s 32us/step
loss= 0.7428267898559571
accuracy= 0.69895
Epoch 1/80
80000/80000 [==============================] - 8s 100us/step - loss: 0.8623 - acc: 0.6319
Epoch 2/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7832 - acc: 0.6574
Epoch 3/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7626 - acc: 0.6627
Epoch 4/80
80000/80000 [==============================] - 7s 94us/step - loss: 0.7533 - acc: 0.6698
Epoch 5/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7447 - acc: 0.6692
Epoch 6/80
80000/80000 [==============================] - 7s 94us/step - loss: 0.7341 - acc: 0.6760
Epoch 7/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7306 - acc: 0.6762
Epoch 8/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7222 - acc: 0.6792
Epoch 9/80
80000/80000 [==============================] - 8s 94us/step - loss: 0.7186 - acc: 0.6

20000/20000 [==============================] - 1s 34us/step
loss= 0.747068425989151
accuracy= 0.70225


In [48]:
temp_01 = []
for i in enumerate(folds.split(lgbm_train01)):
    temp_01.append(i)

vaild_tmp1 = lgbm_train02['label'].iloc[temp_01[0][1][1]]
vaild_tmp2 = lgbm_train02['label'].iloc[temp_01[1][1][1]]
vaild_tmp3 = lgbm_train02['label'].iloc[temp_01[2][1][1]]
vaild_tmp4 = lgbm_train02['label'].iloc[temp_01[3][1][1]]
vaild_tmp5 = lgbm_train02['label'].iloc[temp_01[4][1][1]]

tmps = [vaild_tmp1,vaild_tmp2,vaild_tmp3,vaild_tmp4,vaild_tmp5]


max_pred_avg =  []   

for i in range(len(pred_y)):
    max_pred_avg.append(np.argmax(pred_y[i]))  

comp_lab = pd.DataFrame({'pred':max_pred_avg,'true':tmps[4]})
comp_lab["pred"] = comp_lab["pred"].astype(int)
print('pred:',sum(comp_lab["pred"]==0),sum(comp_lab["pred"]==1),sum(comp_lab["pred"]==2),sum(comp_lab["pred"]==3))
print('true:',sum(comp_lab["true"]==0),sum(comp_lab["true"]==1),sum(comp_lab["true"]==2),sum(comp_lab["true"]==3))

#comp_lab = pd.DataFrame({'pred':max_pred,'true':valid_y})

pr_rc = pd.DataFrame()
for i in range(len(comp_lab['pred'].value_counts())):
    prec = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["pred"][comp_lab["pred"]==i])
    recl = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["true"][comp_lab["true"]==i])
    print("precision : ",prec,", recall : ",recl)
    if i == 0:
        pr_rc = pd.DataFrame({"pred":[prec],"recl":[recl]})
    else:
        pr_rc = pr_rc.append(pd.DataFrame({"pred":[prec],"recl":[recl]}))

f_score = list()
for i in range(len(pr_rc.index)):
    f_score.append(1/(pr_rc["pred"].tolist()[i]))
    f_score.append(1/(pr_rc["recl"].tolist()[i]))
ls_result_lgb = 8/sum(f_score)
print(ls_result_lgb)


pred: 5402 3371 5733 5494
true: 5030 5054 5059 4857
precision :  0.5955201777119585 , recall :  0.6395626242544732
precision :  0.6926727973894987 , recall :  0.462010288880095
precision :  0.7310308738880168 , recall :  0.8284245898398893
precision :  0.783036039315617 , recall :  0.8857319332921556
0.6761301355536679
